In [1]:
# import statements
import numpy as np
import requests
from bs4 import BeautifulSoup, Comment
import time
import csv
from collections import defaultdict
import pickle
import random
import pandas as pd
from util.client import Nba_Season
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

TEAM_NAME_TO_ABR = {
        "ATLANTA HAWKS": 'ATL',
        "BOSTON CELTICS": 'BOS',
        "BROOKLYN NETS": 'BRK',
        "CHARLOTTE HORNETS": 'CHO',
        "CHICAGO BULLS": 'CHI',
        "CLEVELAND CAVALIERS": 'CLE',
        "DALLAS MAVERICKS": 'DAL',
        "DENVER NUGGETS": 'DEN',
        "DETROIT PISTONS": 'DET',
        "GOLDEN STATE WARRIORS": 'GSW',
        "HOUSTON ROCKETS": 'HOU',
        "INDIANA PACERS": 'IND',
        "LOS ANGELES CLIPPERS": 'LAC',
        "LOS ANGELES LAKERS": 'LAL',
        "MEMPHIS GRIZZLIES": 'MEM',
        "MIAMI HEAT": 'MIA',
        "MILWAUKEE BUCKS": 'MIL',
        "MINNESOTA TIMBERWOLVES": 'MIN',
        "NEW ORLEANS PELICANS": 'NOP',
        "NEW YORK KNICKS": 'NYK',
        "OKLAHOMA CITY THUNDER": 'OKC',
        "ORLANDO MAGIC": 'ORL',
        "PHILADELPHIA 76ERS": 'PHI',
        "PHOENIX SUNS": 'PHO',
        "PORTLAND TRAIL BLAZERS": 'POR',
        "SACRAMENTO KINGS": 'SAC',
        "SAN ANTONIO SPURS": 'SAS',
        "TORONTO RAPTORS": 'TOR',
        "UTAH JAZZ": 'UTA',
        "WASHINGTON WIZARDS": 'WAS',

        # DEPRECATED TEAMS
        # "CHARLOTTE BOBCATS": 'CHA',
        # "KANSAS CITY KINGS": 'KCK',
        # "NEW JERSEY NETS": 'NJN',
        # "NEW ORLEANS HORNETS": 'NOH',
        # "NEW ORLEANS/OKLAHOMA CITY HORNETS": 'NOK',
        # "SEATTLE SUPERSONICS": 'SEA',
        # "ST. LOUIS HAWKS": 'STL',
        # "VANCOUVER GRIZZLIES": 'VAN',
        # "WASHINGTON BULLETS": 'WSB',
    }

TEAM_ABR_TO_URL = {
        "ATL": 'https://www.nba.com/stats/team/1610612737',
        "BOS": 'https://www.nba.com/stats/team/1610612738',
        "BRK": 'https://www.nba.com/stats/team/1610612751',
        "CHO": 'https://www.nba.com/stats/team/1610612766',
        "CHI": 'https://www.nba.com/stats/team/1610612741',
        "CLE": 'https://www.nba.com/stats/team/1610612739',
        "DAL": 'https://www.nba.com/stats/team/1610612742',
        "DEN": 'https://www.nba.com/stats/team/1610612743',
        "DET": 'https://www.nba.com/stats/team/1610612765',
        "GSW": 'https://www.nba.com/stats/team/1610612744',
        "HOU": 'https://www.nba.com/stats/team/1610612745',
        "IND": 'https://www.nba.com/stats/team/1610612754',
        "LAC": 'https://www.nba.com/stats/team/1610612746',
        "LAL": 'https://www.nba.com/stats/team/1610612747',
        "MEM": 'https://www.nba.com/stats/team/1610612763',
        "MIA": 'https://www.nba.com/stats/team/1610612748',
        "MIL": 'https://www.nba.com/stats/team/1610612749',
        "MIN": 'https://www.nba.com/stats/team/1610612750',
        "NOP": 'https://www.nba.com/stats/team/1610612740',
        "NYK": 'https://www.nba.com/stats/team/1610612752',
        "OKC": 'https://www.nba.com/stats/team/1610612760',
        "ORL": 'https://www.nba.com/stats/team/1610612753',
        "PHI": 'https://www.nba.com/stats/team/1610612755',
        "PHO": 'https://www.nba.com/stats/team/1610612756',
        "POR": 'https://www.nba.com/stats/team/1610612757',
        "SAC": 'https://www.nba.com/stats/team/1610612758',
        "SAS": 'https://www.nba.com/stats/team/1610612759',
        "TOR": 'https://www.nba.com/stats/team/1610612761',
        "UTA": 'https://www.nba.com/stats/team/1610612762',
        "WAS": 'https://www.nba.com/stats/team/1610612764',
    }

MONTH_TO_NUM = {
        "Jan" : "01",
        "Feb" : "02",
        "Mar" : "03",
        "Apr" : "04",
        "May" : "05",
        "Jun" : "06",
        "Jul" : "07",
        "Aug" : "08",
        "Sep" : "09",
        "Oct" : "10",
        "Nov" : "11",
        "Dec" : "12",
    }

ALL_TEAMS_CONC = dict.fromkeys(TEAM_ABR_TO_URL.keys()) # ABR -> dict(date -> [cum_stats,last10])

## Map teams to NBA stats page

PIE Formula=(PTS + FGM + FTM – FGA – FTA + Deff.REB + Off.REB/2 + AST + STL + BLK/2 – PF – TO) / (Game.PTS + Game.FGM + Game.FTM – Game.FGA – Game.FTA + Game.Deff.REB + Game.Off.REB/2 + Game.AST + Game.STL + Game.BLK/2 – Game.PF – Game.TO)

https://www.nba.com/stats/team/1610612738/traditional?SeasonType=Regular+Season&Split=lastn

https://www.nba.com/stats/team/1610612738/traditional?DateFrom=10%2F24%2F2023&DateTo=10%2F25%2F2023&SeasonType=Regular+Season

https://www.nba.com/stats/team/1610612738/traditional?DateFrom=10%2F24%2F2023&DateTo=3%2F1%2F2024&SeasonType=Regular+Season&Split=lastn

https://www.nba.com/stats/team/1610612738/onoffcourt-traditional?SeasonType=Regular+Season&DateFrom=10%2F24%2F2023&DateTo=1%2F1%2F2024

https://www.nba.com/stats/team/1610612738/players-advanced?SeasonType=Regular+Season&DateFrom=10%2F24%2F2023&DateTo=1%2F1%2F2024 # PIE

Find each teams game dates, get overall stats up to that point and perchance last 10?

In [15]:
def get_cum_stats(dates,base_url):
    '''
    Go through list of dates and populate ALL_TEAMS_CONC
    dates: list of dates similar to `Tue Oct 29 2013`
    base_url: base url of NBA stats page for this team
    abr: team abbreviation
    RETURNS: dict mapping date -> [cum_stats, last10]
    '''
    options = Options()
    chrome_ops = webdriver.ChromeOptions()
    chrome_ops.add_argument("--headless")
    options.headless = True
    driver = webdriver.Chrome(options=options) # requires appropriate webdriver in PATH variables, or pass path to file as arg `executable_path=`
    # driver.implicitly_wait(2)
    wait = WebDriverWait(driver,15)
    out_dict = defaultdict(list)
    missing = []
    i = 0

    for date in dates:
        date_list = date.split()
        team_stats = base_url + f'&DateTo={MONTH_TO_NUM[date_list[1]]}%2F{date_list[2]}%2F{date_list[3]}'
        if i < 82:
            team_stats += '&SeasonType=Regular+Season&Split=lastn'
        else:
            team_stats += '&SeasonType=Playoffs&Split=lastn'
        driver.get(team_stats)
        # load JS
        try:
            wait.until(EC.visibility_of_any_elements_located((By.TAG_NAME, 'table')))
        except Exception as e:
            print(f'URL: {team_stats} caused exception: {e}')
            missing.append(date)
            continue
        
        html = driver.page_source

        soup = BeautifulSoup(html, 'html.parser')
        results = soup.find_all('table')

        # 24 stats total from i = 1 to 24, only care about 3 - 24
        # GP	MIN	PTS	W	L	WIN%	FGM	FGA	FG%	3PM	3PA	3P%	FTM	FTA	FT%	OREB	DREB	REB	AST	TOV	STL	BLK	PF	+/-
        cumalitive = results[2].find_all('td') # cum stats to this point
        cum_stats = []
        for e in cumalitive[3:]: # only care about last ten games?
            cum_stats.append(float(e.text))

        per_n = results[3].find_all('tr') # every 10 games ranges
        last_10 = []

        # if last 10 range includes at least 5 games, use those, else use previous 10 range
        if float(per_n[-1].find_all('td')[1].text) >= 5 or i < 10:
            for e in per_n[-1].find_all('td')[3:]:
                last_10.append(float(e.text))
            
        else:
            for e in per_n[-2].find_all('td')[3:]:
                last_10.append(float(e.text))

        out_dict[date].append(cum_stats)
        out_dict[date].append(last_10)
        i += 1
    
    driver.quit()
    return out_dict,missing

In [2]:
def get_stats_thread(info):
    '''
    Go through list of dates and populate ALL_TEAMS_CONC
    dates: list of dates similar to `Tue Oct 29 2013`
    base_url: base url of NBA stats page for this team
    abr: team abbreviation
    RETURNS: dict mapping date -> [cum_stats, last10]
    '''
    dates = info[0]
    base_url = info[1]
    options = Options()
    chrome_ops = webdriver.ChromeOptions()
    chrome_ops.add_argument("--headless=new")
    options.headless = True
    driver = webdriver.Chrome(options=options) # requires appropriate webdriver in PATH variables, or pass path to file as arg `executable_path=`
    # driver.implicitly_wait(2)
    wait = WebDriverWait(driver,10,0.2)
    out_dict = defaultdict(list)
    missing = []
    i = 0
    try:
        for date in dates:
            date_list = date.split()
            team_stats = base_url + f'&DateTo={MONTH_TO_NUM[date_list[1]]}%2F{date_list[2]}%2F{date_list[3]}'
            if i < 82:
                team_stats += '&SeasonType=Regular+Season&Split=lastn'
            else:
                team_stats += '&SeasonType=Playoffs&Split=lastn'
            driver.get(team_stats)
            # load JS
            try:
                wait.until(EC.visibility_of_all_elements_located((By.TAG_NAME, 'td')))
                # wait.until(EC.visibility_of_any_elements_located((By.TAG_NAME, 'table')))
            except Exception as e:
                # print(f'URL: {team_stats} caused exception: {e}')
                missing.append(date)
                continue
            
            html = driver.page_source

            soup = BeautifulSoup(html, 'html.parser')
            results = soup.find_all('table')

            # 24 stats total from i = 1 to 24, only care about 3 - 24
            # GP	MIN	PTS	W	L	WIN%	FGM	FGA	FG%	3PM	3PA	3P%	FTM	FTA	FT%	OREB	DREB	REB	AST	TOV	STL	BLK	PF	+/-
            cumalitive = results[2].find_all('td') # cum stats to this point
            cum_stats = []
            for e in cumalitive[3:]: # only care about last ten games?
                cum_stats.append(float(e.text))

            per_n = results[3].find_all('tr') # every 10 games ranges
            last_10 = []

            # if last 10 range includes at least 5 games, use those, else use previous 10 range
            if float(per_n[-1].find_all('td')[1].text) >= 5 or i < 10 or len(per_n) < 2:
                for e in per_n[-1].find_all('td')[3:]:
                    last_10.append(float(e.text))
                
            else:
                for e in per_n[-2].find_all('td')[3:]:
                    last_10.append(float(e.text))

            out_dict[date].append(cum_stats)
            out_dict[date].append(last_10)
            i += 1
    except Exception as e:
        driver.quit()
        print(e)
        return info[2],out_dict,missing
    print(f'Data for: {info[2]} complete, Captured: {len(out_dict)}, Missing: {len(missing)}')
    driver.quit()
    return info[2],out_dict,missing

In [3]:
games_df = pd.read_csv('../NBA/games/2023-2024_nba_season.csv',sep=',',names=['date','away','away_pt','home','home_pt'])

In [14]:
import concurrent
import concurrent.futures
# import multiprocessing

def one_szn_par(games_df,szn):
    start_date = games_df['date'][0].split()
    teams = []

    # Generate list of URLS for each team
    for team, abr in TEAM_NAME_TO_ABR.items():
        team_games = games_df.loc[(games_df['away'].str.upper() == team) | (games_df['home'].str.upper() == team)]
        dates = team_games['date'].to_list()
        url = TEAM_ABR_TO_URL[abr] + f'/traditional?Season={szn}&DateFrom={MONTH_TO_NUM[start_date[1]]}%2F{start_date[2]}%2F{start_date[3]}'
        teams.append([dates,url,abr])
        
    with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(get_stats_thread,teams))

    return results

In [13]:
import concurrent
import concurrent.futures
# import multiprocessing

def test_this(games_df,szn):
    start_date = games_df['date'][0].split()
    teams = []

    # Generate list of URLS for each team
    for team, abr in TEAM_NAME_TO_ABR.items():
        team_games = games_df.loc[(games_df['away'].str.upper() == team) | (games_df['home'].str.upper() == team)]
        dates = team_games['date'].to_list()
        url = TEAM_ABR_TO_URL[abr] + f'/traditional?Season={szn}&DateFrom={MONTH_TO_NUM[start_date[1]]}%2F{start_date[2]}%2F{start_date[3]}'
        teams.append([dates[0:1],url,abr])
        
    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
        results = list(executor.map(get_html_thread,teams[0:1]))

    return results

In [14]:
games_df = pd.read_csv('../NBA/games/2023-2024_nba_season.csv',sep=',',names=['date','away','away_pt','home','home_pt'])
results = test_this(games_df,'2023-24')

URL: https://www.nba.com/stats/team/1610612737/traditional?Season=2023-24&DateFrom=10%2F24%2F2023&DateTo=10%2F25%2F2023&SeasonType=Regular+Season&Split=lastn caused exception: Message: 

Data for: ATL complete, Captured: 0, Missing: 1


In [7]:
games_df = pd.read_csv('../NBA/games/2023-2024_nba_season.csv',sep=',',names=['date','away','away_pt','home','home_pt'])
results = test_this(games_df,'2023-24')

Data for: CLE complete, Captured: 0, Missing: 0
Data for: BRK complete, Captured: 0, Missing: 0
Data for: DET complete, Captured: 0, Missing: 0
Data for: DEN complete, Captured: 0, Missing: 0
Data for: CHO complete, Captured: 0, Missing: 0
Data for: CHI complete, Captured: 0, Missing: 0
Data for: GSW complete, Captured: 0, Missing: 1
Data for: BOS complete, Captured: 0, Missing: 1
Data for: DAL complete, Captured: 0, Missing: 1
Data for: ATL complete, Captured: 0, Missing: 0


In [25]:
print('GRUG')

GRUG


In [26]:
abrs = list(TEAM_NAME_TO_ABR.values())

for i in range(len(abrs)):
    # results[i][0]["MISSED"] = results[i][1]
    ALL_TEAMS_CONC[abrs[i]] = results[i][1]
    ALL_TEAMS_CONC[abrs[i]]["MISSED"] = results[i][2]

with open('../NBA/conc_stats/2023-2024_conc_stats.pkl', 'wb') as f:
    pickle.dump(ALL_TEAMS_CONC,f)

In [27]:
with open('../NBA/conc_stats/2023-2024_conc_stats.pkl', 'rb') as f:
    save_me = pickle.load(f)

In [28]:
print('grug')

grug


In [ ]:
start_date = games_df['date'][0].split()
missing = defaultdict(list)

for team, abr in TEAM_NAME_TO_ABR.items():
    print(team)
    team_games = games_df.loc[(games_df['away'].str.upper() == team) | (games_df['home'].str.upper() == team)]
    dates = team_games['date'].to_list()
    url = TEAM_ABR_TO_URL[abr] + f'/traditional?Season=2022-23&DateFrom={MONTH_TO_NUM[start_date[1]]}%2F{start_date[2]}%2F{start_date[3]}' # change ?Season=2022-23& to match rel szn
    res,missed = get_cum_stats(dates,url)
    if len(res) == 0:
        break
    ALL_TEAMS_CONC[abr] = res
    missing[abr] = missed


In [12]:
with open('../NBA/conc_stats/2023-2024_conc_stats.pkl', 'wb') as f:
    pickle.dump(ALL_TEAMS_CONC,f)

In [6]:
ALL_TEAMS_CONC = dict.fromkeys(TEAM_ABR_TO_URL.keys()) # ABR -> dict(date -> [cum_stats,last10])

base_url = 'https://www.nba.com/stats/team/1610612738'
team_stats = base_url + f'/traditional?DateFrom={10}%2F{24}%2F{2023}&DateTo={1}%2F{1}%2F{2024}&SeasonType=Regular+Season&Split=lastn'

options = Options()
options.headless = True
driver = webdriver.Chrome() # requires appropriate webdriver in PATH variables, or pass path to file as arg `executable_path=`
driver.implicitly_wait(10)
wait = WebDriverWait(driver,10)

driver.get(team_stats)

try:
    wait.until(EC.visibility_of(By.CSS_SELECTOR, 'table'))
except Exception as e:
    print(e)

html = driver.page_source
# driver.quit()

soup = BeautifulSoup(html, 'html.parser')
results = soup.find_all('table')

# 24 stats total from i = 1 to 24
# GP	MIN	PTS	W	L	WIN%	FGM	FGA	FG%	3PM	3PA	3P%	FTM	FTA	FT%	OREB	DREB	REB	AST	TOV	STL	BLK	PF	+/-
cumalitive = results[2].find_all('td') # cum stats to this point
cum_stats = []
for e in cumalitive[1:]: # only care about last ten games?
    cum_stats.append(float(e.text))

per_n = results[3].find_all('td') # every 10 games ranges
last_10 = []
for e in per_n[-24:]: # only care about last ten games?
    last_10.append(float(e.text))

driver.quit()

In [13]:
print('greg')

greg


For games 1-10, only use stats up to that point

In [ ]:
def get_stats_to_date(abr,szn_start,date):
    start_month = szn_start[0]
    start_day = szn_start[1]
    start_year = szn_start[2]
    date_list = date.split()
    month = MONTH_TO_NUM[date_list[1]]
    day = date_list[2] if len(date_list[2]) == 2 else "0{day}".format(day=date_list[2])
    year = date_list[3]
    base_url = TEAM_ABR_TO_STATS[abr]

    team_stats = base_url + f'/traditional?DateFrom={start_month}%2F{start_day}%2F{start_year}&DateTo={month}%2F{day}%2F{year}&SeasonType=Regular+Season&Split=lastn'
    page = requests.get(team_stats)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('table')

    return

# self.team_stats[abr]
def gen_stats_per_game(file_path):
    for team,abr in TEAM_NAME_TO_ABR.items():
        team_games = games_df.loc[(games_df['away'].str.upper() == team) | (games_df['home'].str.upper() == team)]
        base_url = TEAM_ABR_TO_STATS[abr]
        

In [4]:
with open('../NBA/on_off_stats/2023-2024_on_off.pkl', 'rb') as f:
    on_off_2023_2024 = pickle.load(f)
with open('../NBA/on_off_stats/2023-2024_team_stats.pkl', 'rb') as f:
    team_stats_2023_2024 = pickle.load(f)
    
nba_szn_2023_2024 = Nba_Season('2023','2024',team_stats=team_stats_2023_2024,team_on_off=on_off_2023_2024)
inj = {'DAL': ['OLIVIER-MAXENCE PROSPER'], 'BOS': []}
home_stats,away_stats = nba_szn_2023_2024.calc_injury_impact(inj, 'BOS', 'DAL')
greg = np.subtract(away_stats,home_stats)

In [5]:
nba_szn_2023_2024.add_bet_info('test_today.csv','test_today_with_bet.csv')

[['Thu Jun 6 2024',
  'Dallas Mavericks',
  '107',
  'Boston Celtics',
  '119',
  '-6.5 (-106)',
  'o214.5 (-112)',
  '-250',
  '+6.5 (-114)',
  'u214.5 (-108)',
  '+205']]